In [1]:
#Import our libraries
import time
import requests
import json
import xml.etree.ElementTree as ET  # for parsing XML
import xmltodict
import pandas as pd
import csv
import base64 
import sqlalchemy

#Import our helper functions
import helper_functions

# Step 1: View the list of reports available

In [2]:
report_lst = helper_functions.get_available_reports()

In [3]:
#Turn it into a list to see all of them
list(report_lst['ReportName'])

['_Current Employee Deductions and Benefits',
 '2013-03-05T21:47:40.188Z',
 '2020-10-26T18:25:32.678Z',
 '2021 Allocation Report (1)',
 '401k Allocation Report V1',
 '401k Allocation Report V2',
 '401k Allocation Report V2_UKG',
 '401k Allocation Report V3_UKG',
 '401k Allocation Report V3_UKG v2',
 '401k Allocation Report V3_UKG v4',
 '5 Level Supervisors',
 'Active EE_DD List',
 'Allocated Hours FTE by Date',
 'Allocated Hours FTE by Month',
 'Annual Earnings',
 'Audit Report with Time vs Pay Matrix Tables',
 'Comp Project V3',
 'Configuration Validation',
 'Counts by Location',
 'Current Employee Deductions and Benefits',
 'Current Employee Deductions and Benefits v2',
 'Current Employee Deductions and Benefits.TEST',
 "David's Report",
 "David's Report V2",
 'Deduction list',
 'Deduction Pay History By Deduction For EmployeesUKG',
 'Deductions in Arrears V2',
 'Deferred Comp Wages and Hours_UKG',
 'Dependents by Employee and Plan test',
 'Document Category Report',
 'DSHS_New Hires

In [4]:
#Get the precheck deduction report pathname
report_lst[report_lst['ReportName'].str.contains('pre', regex=True, case=False)]

,ReportName,ReportPath
64,Pre-Check Deduction,/content/folder[@name='zzzCompany Folders']/fo...
65,Pre-Check Earnings,/content/folder[@name='zzzCompany Folders']/fo...
66,Pre-Check Master,/content/folder[@name='zzzCompany Folders']/fo...
67,Pre-Check Payroll Register,/content/folder[@name='zzzCompany Folders']/fo...
68,Pre-Check Tax,/content/folder[@name='zzzCompany Folders']/fo...
69,Prenote,/content/folder[@name='zzzCompany Folders']/fo...


In [5]:
report_path = report_lst[report_lst['ReportName'] == 'Pre-Check Master']['ReportPath'].loc[66]
report_path

"/content/folder[@name='zzzCompany Folders']/folder[@name='Proliance Surgeons, Inc., P.S.']/folder[@name='UltiPro']/folder[@name='Export to SQL']/report[@name='Pre-Check Master']"

## Step 2: Retrieve the report

In [6]:
df = helper_functions.retrieve_report(report_path) 
df

c:\UKG\Reports as a service\Precheck\helper_functions.py:367: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].astype(str).str.replace('{', '')
c:\UKG\Reports as a service\Precheck\helper_functions.py:368: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].astype(str).str.replace('}', '')
c:\UKG\Reports as a service\Precheck\helper_functions.py:369: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].astype(str).str.replace('[', '')
c:\UKG\Reports as a service\Precheck\helper_funct

,period_control_date,period_control,period_end_date,employee_number,"last,_first_mi",location_code,org_level_1_code,org_level_2_code,hourly_pay_rate,annual_salary,job_code,job_title,total_hours,check_amount,direct_deposit_amount,net_amount,total_earning_amount,total_deduction_amount,total_taxes
0,2023-01-20T00:00:00,202301201,2023-01-14T00:00:00,000310305,Locknane_ Timothy D.,003000,003000,001,57.692308,120000,300,Physician,0,2241.71,0,2241.71,3090.46,463.57,385.18
1,2023-01-20T00:00:00,202301201,2023-01-14T00:00:00,001008227,Torgessen_ Imma L.,000600,000600,001,21,87360,110101,Patient Services Rep,29.15,1107.62,0,1107.62,612.15,-553.81,58.34
2,2023-01-20T00:00:00,202301201,2023-01-14T00:00:00,001008557,Perez_ Christa Lynn M.,008300,008300,001,21,43680,110101,Patient Services Rep,80.17,1035.73,0,1035.73,1191.54,62.5,93.31


In [7]:
#Clean the columns
df['period_control_date'] = df['period_control_date'].str.replace('T', ' ')
df['period_end_date'] = df['period_end_date'].str.replace('T', ' ')
df.rename(columns={'last,_first_mi': 'last_first_mi'}, inplace=True)
df

,period_control_date,period_control,period_end_date,employee_number,last_first_mi,location_code,org_level_1_code,org_level_2_code,hourly_pay_rate,annual_salary,job_code,job_title,total_hours,check_amount,direct_deposit_amount,net_amount,total_earning_amount,total_deduction_amount,total_taxes
0,2023-01-20 00:00:00,202301201,2023-01-14 00:00:00,000310305,Locknane_ Timothy D.,003000,003000,001,57.692308,120000,300,Physician,0,2241.71,0,2241.71,3090.46,463.57,385.18
1,2023-01-20 00:00:00,202301201,2023-01-14 00:00:00,001008227,Torgessen_ Imma L.,000600,000600,001,21,87360,110101,Patient Services Rep,29.15,1107.62,0,1107.62,612.15,-553.81,58.34
2,2023-01-20 00:00:00,202301201,2023-01-14 00:00:00,001008557,Perez_ Christa Lynn M.,008300,008300,001,21,43680,110101,Patient Services Rep,80.17,1035.73,0,1035.73,1191.54,62.5,93.31


# Step 3: Export the report into a CSV

In [8]:
df.to_csv('ukg_bulk_insert.csv', index=False)

# Step 4: Insert the report into BiData2 as a table

In [9]:
helper_functions.create_sql_table_v2('STG_PreCheck_Master')

# Step 5: Close the API connection

In [10]:
helper_functions.close_api_connection()

'<s:Envelope xmlns:s="http://www.w3.org/2003/05/soap-envelope" xmlns:a="http://www.w3.org/2005/08/addressing"><s:Header><a:Action s:mustUnderstand="1">http://www.ultipro.com/dataservices/bidata/2/IBIDataService/LogOffResponse</a:Action></s:Header><s:Body><LogOffResponse xmlns="http://www.ultipro.com/dataservices/bidata/2"><LogOffResult xmlns:i="http://www.w3.org/2001/XMLSchema-instance"><ServiceId i:nil="true"/><ClientAccessKey>PLAB4</ClientAccessKey><Token i:nil="true"/><Status>LoggedOff</Status><StatusMessage i:nil="true"/><InstanceKey i:nil="true"/></LogOffResult></LogOffResponse></s:Body></s:Envelope>'